In [ ]:
# Install Ultralytics YOLO package
!pip install ultralytics

# Install OpenCV for image processing
!pip install opencv-python-headless

# Matplotlib is usually pre-installed in Colab, but you can install it if needed
!pip install matplotlib
# OpenCV for image processing

# Pillow (PIL) for image handling
!pip install Pillow

# NumPy (usually pre-installed on Colab)
!pip install numpy

# IPython for interactive widgets and display (should already be installed)
!pip install ipython
# Install PaddleOCR and PaddlePaddle
!pip install paddlepaddle paddleocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.6/544.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.8 MB/s eta 0:00:00
   ━━━

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import cv2
import PIL
from io import BytesIO
from google.colab.patches import cv2_imshow

# JavaScript to display the webcam feed and add a 'Capture' button
def js_capture_image():
    js = Javascript('''
    async function captureImage() {
        // Check if video and button elements already exist, if so, remove them
        const existingVideo = document.querySelector('video');
        const existingButton = document.querySelector('button');
        if (existingVideo) existingVideo.remove();
        if (existingButton) existingButton.remove();

        // Create video and button elements
        const video = document.createElement('video');
        const button = document.createElement('button');
        button.textContent = 'Capture';

        // Append video and button to the page
        document.body.appendChild(video);
        document.body.appendChild(button);

        // Set up webcam stream
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        video.srcObject = stream;
        video.style.width = '320px';
        await new Promise((resolve) => video.onloadedmetadata = resolve);
        video.play();

        // Wait for the 'Capture' button to be clicked
        await new Promise((resolve) => button.onclick = resolve);

        // Capture the image from the video stream
        const canvas = document.createElement('canvas');
        const context = canvas.getContext('2d');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        context.drawImage(video, 0, 0, canvas.width, canvas.height);

        // Stop the webcam and remove the video and button elements
        stream.getTracks().forEach(track => track.stop());
        video.remove();
        button.remove();

        // Return the captured image data
        return canvas.toDataURL('image/png');
    }
    captureImage();
    ''')
    display(js)

# Function to decode image and convert it to OpenCV format
def get_image():
    image_data = eval_js('captureImage()')
    image_bytes = b64decode(image_data.split(',')[1])
    image = np.array(PIL.Image.open(BytesIO(image_bytes)))
    return image

# Trigger the capture image process
js_capture_image()
image = get_image()

# Show the captured image
cv2_imshow(image)

# Save the image to /content/ (Colab file explorer section)
image_path = '/content/captured_image.png'
cv2.imwrite(image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

# Display a message confirming the saved image path
print(f"Image saved to: {image_path}")

<IPython.core.display.Javascript object>

In [ ]:
from datetime import datetime
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from paddleocr import PaddleOCR
import re

# Initialize PaddleOCR model
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Set language as English

# Step 1: Load the YOLO model (either pretrained or your trained model)
model_trained = YOLO('/content/expmrp (1).pt')  # Path to your trained model

# Step 2: Define the path to the test image
image_path = '/content/captured_image.png'

# Step 3: Perform inference using the trained YOLO model
results_trained = model_trained.predict(source=image_path, conf=0.25)

# Step 4: Extract brand name text using PaddleOCR from YOLO bounding boxes
def extract_text_from_boxes(image_path, results):
    # Load the image
    img = cv2.imread(image_path)

    # Check if the image is loaded properly
    if img is None:
        raise ValueError(f"Could not load image from path: {image_path}")

    extracted_texts = []  # Store extracted text from each bounding box

    # Iterate over the detected objects and their bounding boxes
    for result in results:
        # Assuming results contain bounding box information
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates

            # Ensure the coordinates are within the image dimensions
            x1, y1, x2, y2 = max(0, x1), max(0, y1), min(img.shape[1], x2), min(img.shape[0], y2)

            # Crop the bounding box area from the image
            cropped_img = img[y1:y2, x1:x2]

            # Use PaddleOCR to detect text in the cropped area
            ocr_result = ocr.ocr(cropped_img, cls=True)

            # Extract detected text and append to the list
            for line in ocr_result[0]:
                extracted_texts.append(line[1][0])  # Extracting the text part

    return extracted_texts

# Step 5: Extract text from bounding boxes
extracted_texts = extract_text_from_boxes(image_path, results_trained)

# Step 6: Extract RS, Manufacture Date, and Expiry Date using regex
def extract_info_from_text(text_list):
    # Updated Regex Patterns
    rs_pattern = r'(?:Rs\.?\s*\d+(\.\d{1,2})?\s?/?\w{0,2}|(?:₹|\bRs\.?|INR)?\s*\d+(?:\.\d{1,2})?)'

    # Date pattern to capture multiple formats like dd/mm/yyyy, dd.mm.yy, dd-mm-yy, etc.
    date_pattern = r'(\d{2}[./-]\d{2}[./-]\d{4}|\d{2}[./-]\d{2}[./-]\d{2}|\d{2}/\d{4}|\d{2}/\d{2}/\d{4})'

    rs_value = None
    dates = []

    for text in text_list:
        # Find RS value
        if not rs_value:
            rs_match = re.search(rs_pattern, text)
            if rs_match:
                rs_value = rs_match.group()

        # Find all dates
        date_matches = re.findall(date_pattern, text)
        if date_matches:
            dates.extend([m[0] if isinstance(m, tuple) else m for m in date_matches])

    # Sort dates to identify manufacture and expiry dates
    if dates:
        sorted_dates = sorted(dates, key=lambda d: [int(x) for x in re.split(r'[./-]', d) if x.isdigit()][::-1])  # Sort dates by year, month, day
        manufacture_date = sorted_dates[0]  # Earliest date
        expiry_date = sorted_dates[-1]  # Latest date
    else:
        manufacture_date = expiry_date = None

    return rs_value, manufacture_date, expiry_date

# Call the function to extract RS, Manufacture, and Expiry Date
rs_value, manufacture_date, expiry_date = extract_info_from_text(extracted_texts)

# Get current date
current_date = datetime.now().strftime("%d/%m/%Y")

# Compare expiry date with the current date
def compare_dates(expiry_date_str, current_date_str):
    # Define the list of date formats to try
    date_formats = ["%d/%m/%Y", "%d-%m-%Y", "%d.%m.%Y", "%d/%m/%y", "%d-%m-%y", "%d.%m.%y"]

    for date_format in date_formats:
        try:
            expiry_date_obj = datetime.strptime(expiry_date_str, date_format)
            current_date_obj = datetime.strptime(current_date_str, "%d/%m/%Y")

            # Compare the expiry date and the current date
            if current_date_obj <= expiry_date_obj:
                return "Good"
            else:
                return "Bad"
        except ValueError:
            continue

    # If no format matches, return an error message
    return "Error parsing expiry date"

# Print the extracted text and extracted information
print("Extracted Text from Bounding Boxes:")
for text in extracted_texts:
    print(text)

# Display the extracted RS, manufacture date, expiry date, and current date
print("\nExtracted Information:")
print(f"MRP Value: {rs_value}")
print(f"Manufacture Date: {manufacture_date}")
print(f"Expiry Date: {expiry_date}")
print(f"Current Date: {current_date}")

# Check if expiry date is valid and compare it with the current date
if expiry_date:
    result = compare_dates(expiry_date, current_date)
    print(f"Status: {result}")
else:
    print("Expiry date not found.")

# Optional: Visualize the YOLO detection with bounding boxes
def visualize_results(image_path, results, title=""):
    # Load the image using OpenCV
    img = cv2.imread(image_path)

    # Check if the image is loaded properly
    if img is None:
        raise ValueError(f"Could not load image from path: {image_path}")

    # Draw bounding boxes and labels
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            conf = box.conf[0]  # Confidence score
            label = f'{result.names[int(box.cls[0])]} {conf:.2f}'  # Class label and confidence

            # Draw rectangle and put the label on the image
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Convert image to RGB for displaying in matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Display the image
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis('off')  # Hide axes
    plt.show()

# Visualize the YOLO results (Optional)
visualize_results(image_path, results_trained, title="Trained Model Results with Bounding Boxes")